In [1]:
import os
import pandas as pd
from Bio import SeqIO
import gzip
import os

genome_dir = "data/zebrafishGenome"
chr_file = os.path.join(genome_dir, f"Danio_rerio.GRCz11.dna.chromosome.1.fa.gz")

with gzip.open(chr_file, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        # ensembl_havana	start_codon	22834683	22834685	.	-	0	gene_id "ENSDARG00000034896";
        seq = record.seq[22834683-1:22834685] # start - ENSDARG00000034896
        print("- start", seq)
        seq = record.seq[22834683-5:22834685+5] # start - ENSDARG00000034896
        print("- start", seq)
        # ensembl_havana	stop_codon	22808685	22808687	.	-	0	gene_id "ENSDARG00000034896"
        seq = record.seq[22808685-1:22808687]
        print("- end", seq)
        seq = record.seq[22808685-5:22808687+5]
        print("- end", seq)

        # havana	start_codon	49652967	49652969	.	+	0	gene_id "ENSDARG00000052512"
        seq = record.seq[49652967-1:49652969] # start + ENSDARG00000052512
        print("+ start", seq)
        seq = record.seq[49652967-5:49652969+5] # start + ENSDARG00000052512
        print("+ start", seq)
        # havana	stop_codon	49673886	49673888	.	+	0	gene_id "ENSDARG00000052512"
        seq = record.seq[49673886-1:49673888] # start + ENSDARG00000052512
        print("+ end", seq)
        seq = record.seq[49673886-5:49673888+5] # start + ENSDARG00000052512
        print("+ end", seq)

        # ensembl	start_codon	49653487	49653489	.	+	0	gene_id "ENSDARG00000052512"
        seq = record.seq[49653487-1:49653489] # start + ENSDARG00000052512
        print("+ start", seq)
        seq = record.seq[49653487-5:49653489+5] # start + ENSDARG00000052512
        print("+ start", seq)
        # ensembl	stop_codon	49673886	49673888	.	+	0	gene_id "ENSDARG00000052512"
        seq = record.seq[49673886-1:49673888] # start + ENSDARG00000052512
        print("+ end", seq)
        seq = record.seq[49673886-5:49673888+5] # start + ENSDARG00000052512
        print("+ end", seq)

        # ensembl_havana	start_codon	32528097	32528099	.	+	0	gene_id "ENSDARG00000079455"
        seq = record.seq[32528097-1:32528099]
        print("+ start", seq)
        seq = record.seq[32528097-5:32528099+5]
        print("+ start", seq)
        # ensembl_havana	stop_codon	32691704	32691706	.	+	0	gene_id "ENSDARG00000079455"
        seq = record.seq[32691704-1:32691706]
        print("+ end", seq)
        seq = record.seq[32691704-5:32691706+5]
        print("+ end", seq)

        # havana	start_codon	24255147	24255149	.	-	0	gene_id "ENSDARG00000031108"
        seq = record.seq[24255147-1:24255149]
        print("- start", seq)
        seq = record.seq[24255147-5:24255149+5]
        print("- start", seq)

- start CAT
- start CGGTCATGGTTT
- end TTA
- end CACATTACTCCA
+ start ATG
+ start AGAGATGTTGAG
+ end TGA
+ end CGACTGAGACAC
+ start ATG
+ start AAACATGAGCGG
+ end TGA
+ end CGACTGAGACAC
+ start ATG
+ start GACCATGTCACG
+ end TAG
+ end GGTATAGCTTCT
- start CAT
- start AACACATAAAAA


In [8]:
from util.gtf2df import gtf2df
import pandas as pd
from Bio import SeqIO
import gzip
import os

genome_dir = "data/zebrafishGenome"
gtf_file = "data\\Danio_rerio.GRCz11.113.gtf"
df = gtf2df(gtf_file)
df_gene_ids = pd.read_csv("data/all_values.txt", sep=",")["Gene ID"]
counter = 0
for id in df_gene_ids:
    counter += 1
    if counter % 100 == 0:
        print("at counter", counter)

    gene_rows = df[df["gene_id"] == id]
    seq_id = gene_rows.iloc[0]["seqname"]
    chr_file = os.path.join(genome_dir, f"Danio_rerio.GRCz11.dna.chromosome.{seq_id}.fa.gz")
    with gzip.open(chr_file, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            start_codon_rows = df[(df["gene_id"] == id) & (df["feature"] == "start_codon")]
            stop_codon_rows = df[(df["gene_id"] == id) & (df["feature"] == "stop_codon")]
            if start_codon_rows.empty or stop_codon_rows.empty:
                continue

            start_codon_row = start_codon_rows.iloc[0]
            stop_codon_row = stop_codon_rows.iloc[0]
            start_codon_start, start_codon_end = (start_codon_row.loc["start"], start_codon_row.loc["end"])
            stop_codon_start, stop_codon_end = (stop_codon_row.loc["start"], stop_codon_row.loc["end"])
            strand = start_codon_row.loc["strand"]
            if strand == "+":
                print("For gene", id, "with strand", strand, "and start codon coordinates", start_codon_start, "and stop codon coordinates", stop_codon_start)
                print("Is start codon coordinate smaller than the stop codon coordinate?", (start_codon_start < stop_codon_start))
                start_codon = record.seq[start_codon_start-1:start_codon_end]
                print("Starting codon", start_codon, "in context", record.seq[start_codon_start-6:start_codon_end+5])
                stop_codon = record.seq[stop_codon_start-1:stop_codon_end] # TAA, TAG, and TGA
                print("Stopping codon", stop_codon, "in context", record.seq[stop_codon_start-6:stop_codon_end+5])
                print("")
            else:
                print("For gene", id, "with strand", strand, "and start codon coordinates", start_codon_start, "and stop codon coordinates", stop_codon_start)
                print("Is start codon coordinate smaller than the stop codon coordinate?", (start_codon_start < stop_codon_start))
                start_codon = record.seq[start_codon_start-1:start_codon_end]
                print("Starting codon", start_codon.reverse_complement(), "in context", record.seq[start_codon_start-6:start_codon_end+5].reverse_complement())
                stop_codon = record.seq[stop_codon_start-1:stop_codon_end] # TAA, TAG, and TGA
                print("Stopping codon", stop_codon.reverse_complement(), "in context", record.seq[stop_codon_start-6:stop_codon_end+5].reverse_complement())
                #print("IN 34111839-34111839+10")
                #print("Nothing:", record.seq[34111839:34111839+10])
                #print("Reverse:", record.seq[34111839:34111839+10][::-1])
                #print("Reverse.complement:", record.seq[34111839:34111839+10].reverse_complement())
                print("")

C:\Users\gusta\PycharmProjects\ThesisPlayground\util\gtf2df.py:27: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(gtf, sep='\t', header=None, comment='#')


For gene ENSDARG00000000001 with strand - and start codon coordinates 34120903 and stop codon coordinates 34114749
Is start codon coordinate smaller than the stop codon coordinate? False
Starting codon ATG in context AGCACATGTCCTT
Stopping codon TGA in context CTTTCTGAGTGGG
IN 34111839-34111839+10
Nothing: AAAAATATTG
Reverse: GTTATAAAAA
Reverse.complement: CAATATTTTT


KeyboardInterrupt: 